In [4]:
from __future__ import print_function, absolute_import, unicode_literals

import MySQLdb
import codecs
from datetime import *
import time
import json

import sys
sys.path.append('../')
from DB import *
from IndexConstituentManager import *
from StockService import *

class PEManager:
    def __init__(self):
        self.db = DB()
        self.indexConstituent = IndexConstituentManager()
        self.stockService = StockService()
        self.tradeDates = []
        self._loadTradeDates()
    
    def _loadTradeDates(self):
        sql = "SELECT end_date FROM trading_derivative_indicator WHERE code = '000001'"
        results = self.db.executeSql(sql)
        for item in results:
            self.tradeDates.append(self._dateToStr(item[0]))
    
    def isTradeDate(self, date):
        return self._dateToStr(date) in self.tradeDates
    
    def _dateToStr(self, date):
        return date.strftime("%Y-%m-%d")
    
    def getProfitAt(self, indexCode, date):
        sql = "SELECT code, pub_date, end_date, NETPROFIT FROM income_statement WHERE code = '%s' and pub_date <= '%s' ORDER BY pub_date LIMIT 1"
        sql = sql % (indexCode, date.strftime("%Y-%m-%d"))
        result = self.db.executeSql(sql)
        if len(result) > 0:
            return {"code" : result[0][0], "pub_date" : result[0][1], "end_date" : result[0][2], "net_profit" : result[0][3]}
    
    def getLatestDateOfField(self, indexCode, field):
        sql = "SELECT pub_date FROM index_derivative_indicator WHERE code = '%s' and %s is not null"
        sql = sql % (indexCode, field)
        result = self.db.executeSql(sql)
        if len(result) == 0:
            return datetime(2005,1,4)
        else:
            return result[0][0]
    
    def equalWeightedPE(self, indexCode, startDate = None):
        date = startDate == None ? datetime(2005,1,4) : startDate
        now = datetime.now()
        equalWeightPE = {}
        while date <= now:
            if self.isTradeDate(date):
                constituents = self.indexConstituent.getConstituents(indexCode, date)
                if constituents is not None:
                    stockPETTM = [self.stockService.getStockPETTM(stock, date) for stock in constituents]
                    pe = len(stockPETTM)/sum([1/p if p>0 else 0 for p in stockPETTM])
#                     print(date.date, pe)
                    equalWeightPE[self._dateToStr(date)] = pe
                else:
                    print("error", indexCode, date)
            date = date + timedelta(days=1)
        return equalWeightPE
    
    def updateAllByIndex(self, indexCode):
        peManager.equalWeightedPE("000913")
        
peManager = PEManager()
peManager.equalWeightedPE("000913")
# print(peManager.tradeDates)

        
        

Load stock 600085 PETTM successfully
Load stock 000623 PETTM successfully
Load stock 600812 PETTM successfully
Load stock 600196 PETTM successfully
Load stock 600062 PETTM successfully
Load stock 000538 PETTM successfully
Load stock 600535 PETTM successfully
Load stock 600161 PETTM successfully
Load stock 600521 PETTM successfully
Load stock 600380 PETTM successfully
Load stock 000423 PETTM successfully
Load stock 002001 PETTM successfully
Load stock 600664 PETTM successfully
Load stock 600518 PETTM successfully
Load stock 000999 PETTM successfully
Load stock 002038 PETTM successfully
Load stock 600276 PETTM successfully
Load stock 600216 PETTM successfully
Load stock 002275 PETTM successfully
Load stock 002007 PETTM successfully
Load stock 002294 PETTM successfully
Load stock 002399 PETTM successfully
Load stock 002422 PETTM successfully
Load stock 600267 PETTM successfully
Load stock 600252 PETTM successfully
Load stock 002603 PETTM successfully
Load stock 600436 PETTM successfully
L

{'2005-01-04': 27.44936249800841,
 '2005-01-05': 27.649857017006802,
 '2005-01-06': 27.75592655086934,
 '2005-01-07': 27.964227585755708,
 '2005-01-10': 28.371908341847487,
 '2005-01-11': 28.32197019879802,
 '2005-01-12': 28.242575399791964,
 '2005-01-13': 28.203396718848563,
 '2005-01-14': 28.242399160372877,
 '2005-01-17': 27.742146206444843,
 '2005-01-18': 27.956730954284957,
 '2005-01-19': 27.83278141366279,
 '2005-01-20': 27.61248418850466,
 '2005-01-21': 28.11770977040495,
 '2005-01-24': 28.497645975826774,
 '2005-01-25': 28.391417913524847,
 '2005-01-26': 28.364115443909842,
 '2005-01-27': 28.211429259650572,
 '2005-01-28': 27.82569434293799,
 '2005-01-31': 28.726394177324675,
 '2005-02-01': 28.362743980189265,
 '2005-02-02': 29.45824595844149,
 '2005-02-03': 29.310874008117384,
 '2005-02-04': 29.855741164292635,
 '2005-02-16': 30.073597038966785,
 '2005-02-17': 30.197191278026146,
 '2005-02-18': 29.799990120447642,
 '2005-02-21': 30.072783695331985,
 '2005-02-22': 30.4956204091